In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('All_Data.csv', sep=';')

# list_of_feats_Causal_Discovery = ['MD 210',
#  'DB 70DBD 4 ΘΕΡΜ ΚΟΛΕΚΤΕΡ ΨΥΓΕΙΩΝ (ΝΕΡΟ ΑΠΟ ΔΕΞΑΜΕΝΗ)',
#  'DB 30DBW 20 ramposition',
#  'DB 20DBW 174 EXTRACTION STEP',
#  'DB 10DBW 14 BACKWARD PRESS',
#  'DB 20DBD 292',
#  'Setpoint position exhaust damper',
#  'DB 400DBD 34 z2 energy',
#  'T 174',
#  'MD 284 C ana kw',
#  'MD 70 sinolo ypog-mpanioy', 'Timestamp', 'Energy']

# df = df[list_of_feats_Causal_Discovery]
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df[:-120] #cutoff used in the training pipeline as well
df

,Timestamp,DB 10DBW 14 BACKWARD PRESS,DB 1260DBD 100 TOOL SLIDE POSITION,DB 13DBW 2,DB 30DBW 140 DISCHARGER POSITION,DB 30DBW 20 ramposition,DB 30DBW 80 v cont position,DB 371DBW 44 setpoint pump,MW 970,DB 10DBW 232,...,DB 75DBD 28,MD 210,MD 214,MD 70 sinolo ypog-mpanioy,T 0,C 22_Junker2,C 23_Junker2,C 24_Junker2,C 25_Junker2,Energy
0,2025-02-28 13:00:00,5.800000,635.218667,381.000000,2.2,1223.400000,20.133333,891.733333,15.800000,360.000000,...,7.266667,350.000000,450.000000,2736.063333,0.0,4.0,1.0,5.000000,2.0,101.671
1,2025-02-28 13:15:00,6.266667,508.268667,196.666667,52.8,1215.466667,15.533333,731.266667,15.866667,168.733333,...,1.600000,496.666667,716.666667,2731.106667,0.0,4.0,1.0,5.000000,2.0,153.854
2,2025-02-28 13:30:00,9.600000,381.356667,241.800000,52.8,803.866667,11.866667,826.933333,14.933333,108.666667,...,2.200000,146.666667,533.333333,2726.779333,0.0,4.0,1.0,5.000000,2.0,134.813
3,2025-02-28 13:45:00,9.133333,381.372000,168.466667,1.0,1031.133333,11.866667,830.400000,15.333333,123.133333,...,6.000000,0.000000,266.666667,2726.767333,0.0,4.0,1.0,5.000000,2.0,145.956
4,2025-02-28 14:00:00,6.600000,667.294667,-105.333333,104.6,954.800000,25.000000,943.466667,15.066667,169.000000,...,9.666667,293.333333,666.666667,2725.181333,0.0,4.0,1.0,4.466667,2.0,140.812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166,2025-03-23 02:30:00,0.000000,1904.660000,0.000000,1.0,1.000000,505.000000,989.733333,10.000000,32.000000,...,0.000000,0.000000,0.000000,2783.607333,0.0,1.0,1.0,3.000000,1.0,14.997
2167,2025-03-23 02:45:00,0.000000,1904.660000,0.000000,1.0,1.000000,505.000000,986.466667,10.000000,32.000000,...,0.000000,0.000000,0.000000,2783.610000,0.0,1.0,1.0,3.000000,1.0,5.174
2168,2025-03-23 03:00:00,0.000000,1904.660000,0.000000,1.0,1.000000,505.000000,982.066667,10.000000,32.000000,...,0.000000,0.000000,0.000000,2783.237333,0.0,1.0,1.0,3.000000,1.0,5.886
2169,2025-03-23 03:15:00,0.000000,1904.660000,0.000000,1.0,1.000000,505.000000,979.866667,10.000000,32.000000,...,0.000000,0.000000,0.000000,2783.213333,0.0,1.0,1.0,3.000000,1.0,12.474


In [2]:
 # Categorical features
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)
df['dow'] = df.index.day_name().str[:3]     # e.g. "Mon","Tue",…,"Sun"
df['hour'] = df.index.hour # 0–23
sun_low = df['dow'] == 'Sun' # basically all day on sunday
mon_low = (df['dow'] == 'Mon') & df['hour'].isin([0,1,2,3,4,5,6,7,8,9]) # basically from 0 to 9 on monday
wed_low = (df['dow'] == 'Wed') & df['hour'].isin([6,7,8,9,10,11,12,13,14]) # basically from 6 to 14 for maintenance on wednesday
sat_low = (df['dow'] == 'Sat') & df['hour'].isin([19,20,21,22,23]) # basically from 19 to 23 on saturday
df['is_low_usage'] = sun_low | mon_low | wed_low | sat_low
df.drop(columns=['dow', 'hour'], inplace=True)
df.reset_index(inplace=True)
df['is_low_usage'] = df['is_low_usage'].astype(int)
df['is_low_usage_next'] = df['is_low_usage'].shift(-1)




# cyclical encode timestamp as extra features
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')
df.set_index('Timestamp', inplace=True)
df['dow'] = df.index.day_name().str[:3]     # e.g. "Mon","Tue",…,"Sun"
df['hour'] = df.index.hour # 0–23
sun_low = df['dow'] == 'Sun' # basically all day on sunday
mon_low = (df['dow'] == 'Mon') & df['hour'].isin([0,1,2,3,4,5,6,7,8,9]) # basically from 0 to 9 on monday
wed_low = (df['dow'] == 'Wed') & df['hour'].isin([6,7,8,9,10,11,12,13,14]) # basically from 6 to 14 for maintenance on wednesday
sat_low = (df['dow'] == 'Sat') & df['hour'].isin([19,20,21,22,23]) # basically from 19 to 23 on saturday
df['is_low_usage'] = sun_low | mon_low | wed_low | sat_low
df.drop(columns=['dow', 'hour'], inplace=True)
df.reset_index(inplace=True)
df['is_low_usage'] = df['is_low_usage'].astype(int)



# Hour of day (0–23)
df['hour_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.hour / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.hour / 24)

# # Minute of hour (0–59)
# df['minute_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.minute / 60)
# df['minute_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.minute / 60)

# Compute number of days in each month
df['days_in_month'] = df['Timestamp'].dt.days_in_month
df['day_of_week_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.dayofweek / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.dayofweek / 7)
# Safe cyclic encoding
# df['day_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.day / df['days_in_month'])
# df['day_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.day / df['days_in_month'])
# Categorical features

# df['month_sin'] = np.sin(2 * np.pi * df['Timestamp'].dt.month / 12)
# df['month_cos'] = np.cos(2 * np.pi * df['Timestamp'].dt.month / 12)
df.drop(columns=['Timestamp', 'days_in_month'], inplace=True)
df.to_csv('energy_data.csv', index=False, sep=',')
df

,DB 10DBW 14 BACKWARD PRESS,DB 1260DBD 100 TOOL SLIDE POSITION,DB 13DBW 2,DB 30DBW 140 DISCHARGER POSITION,DB 30DBW 20 ramposition,DB 30DBW 80 v cont position,DB 371DBW 44 setpoint pump,MW 970,DB 10DBW 232,DB 10DBW 8 billet length,...,C 23_Junker2,C 24_Junker2,C 25_Junker2,Energy,is_low_usage,is_low_usage_next,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos
0,5.800000,635.218667,381.000000,2.2,1223.400000,20.133333,891.733333,15.800000,360.000000,670.0,...,1.0,5.000000,2.0,101.671,0,0.0,-0.258819,-0.965926,-0.433884,-0.900969
1,6.266667,508.268667,196.666667,52.8,1215.466667,15.533333,731.266667,15.866667,168.733333,670.0,...,1.0,5.000000,2.0,153.854,0,0.0,-0.258819,-0.965926,-0.433884,-0.900969
2,9.600000,381.356667,241.800000,52.8,803.866667,11.866667,826.933333,14.933333,108.666667,670.0,...,1.0,5.000000,2.0,134.813,0,0.0,-0.258819,-0.965926,-0.433884,-0.900969
3,9.133333,381.372000,168.466667,1.0,1031.133333,11.866667,830.400000,15.333333,123.133333,670.0,...,1.0,5.000000,2.0,145.956,0,0.0,-0.258819,-0.965926,-0.433884,-0.900969
4,6.600000,667.294667,-105.333333,104.6,954.800000,25.000000,943.466667,15.066667,169.000000,670.0,...,1.0,4.466667,2.0,140.812,0,0.0,-0.500000,-0.866025,-0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166,0.000000,1904.660000,0.000000,1.0,1.000000,505.000000,989.733333,10.000000,32.000000,810.0,...,1.0,3.000000,1.0,14.997,1,1.0,0.500000,0.866025,-0.781831,0.623490
2167,0.000000,1904.660000,0.000000,1.0,1.000000,505.000000,986.466667,10.000000,32.000000,810.0,...,1.0,3.000000,1.0,5.174,1,1.0,0.500000,0.866025,-0.781831,0.623490
2168,0.000000,1904.660000,0.000000,1.0,1.000000,505.000000,982.066667,10.000000,32.000000,810.0,...,1.0,3.000000,1.0,5.886,1,1.0,0.707107,0.707107,-0.781831,0.623490
2169,0.000000,1904.660000,0.000000,1.0,1.000000,505.000000,979.866667,10.000000,32.000000,810.0,...,1.0,3.000000,1.0,12.474,1,1.0,0.707107,0.707107,-0.781831,0.623490


In [3]:
import pandas as pd

df = pd.read_csv('energy_data.csv')
column_to_move = 'Energy'
df = df[[col for col in df.columns if col != column_to_move] + [column_to_move]]
df.to_csv('energy_data.csv', index=False)

In [4]:
len(df.columns)

57

In [5]:
import pandas as pd

df = pd.read_csv('energy_data.csv')
n = len(df)
train_end = int(n * 0.85)
X_train = df.iloc[:train_end]
X_train.to_csv('energy_data.csv', index=False)

In [6]:
len(X_train.columns)

57

In [7]:
X_train

,DB 10DBW 14 BACKWARD PRESS,DB 1260DBD 100 TOOL SLIDE POSITION,DB 13DBW 2,DB 30DBW 140 DISCHARGER POSITION,DB 30DBW 20 ramposition,DB 30DBW 80 v cont position,DB 371DBW 44 setpoint pump,MW 970,DB 10DBW 232,DB 10DBW 8 billet length,...,C 23_Junker2,C 24_Junker2,C 25_Junker2,is_low_usage,is_low_usage_next,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,Energy
0,5.800000,635.218667,381.000000,2.200000,1223.400000,20.133333,891.733333,15.800000,360.000000,670.0,...,1.0,5.000000,2.0,0,0.0,-0.258819,-9.659258e-01,-0.433884,-0.900969,101.671
1,6.266667,508.268667,196.666667,52.800000,1215.466667,15.533333,731.266667,15.866667,168.733333,670.0,...,1.0,5.000000,2.0,0,0.0,-0.258819,-9.659258e-01,-0.433884,-0.900969,153.854
2,9.600000,381.356667,241.800000,52.800000,803.866667,11.866667,826.933333,14.933333,108.666667,670.0,...,1.0,5.000000,2.0,0,0.0,-0.258819,-9.659258e-01,-0.433884,-0.900969,134.813
3,9.133333,381.372000,168.466667,1.000000,1031.133333,11.866667,830.400000,15.333333,123.133333,670.0,...,1.0,5.000000,2.0,0,0.0,-0.258819,-9.659258e-01,-0.433884,-0.900969,145.956
4,6.600000,667.294667,-105.333333,104.600000,954.800000,25.000000,943.466667,15.066667,169.000000,670.0,...,1.0,4.466667,2.0,0,0.0,-0.500000,-8.660254e-01,-0.433884,-0.900969,140.812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,11.666667,254.353333,454.933333,3.466667,928.733333,15.933333,866.866667,11.466667,263.333333,810.0,...,1.0,4.000000,3.0,0,0.0,-0.965926,-2.588190e-01,0.974928,-0.222521,165.597
1841,3.400000,455.794000,43.333333,75.800000,863.800000,15.533333,911.000000,11.866667,264.066667,810.0,...,1.0,4.000000,3.0,0,0.0,-0.965926,-2.588190e-01,0.974928,-0.222521,179.747
1842,6.933333,402.017333,-3.466667,52.800000,1167.400000,24.000000,852.666667,12.200000,125.200000,810.0,...,1.0,4.000000,3.0,0,0.0,-0.965926,-2.588190e-01,0.974928,-0.222521,163.784
1843,4.733333,254.433333,1356.600000,52.800000,1150.333333,6.800000,905.600000,11.600000,217.333333,810.0,...,1.0,4.000000,3.0,0,0.0,-0.965926,-2.588190e-01,0.974928,-0.222521,118.536
